# Solution One
1. First field: "DATAFLOW" TEXT  
I will ignore now because I believe that flows do not interact 
so will have different tables (or even databases) for each flow
(the flows separation hipotesys must be verified with the owner)

2. These are the fields that will form the key (__group1__)

  "FREQ" TEXT,  
  "TIME_PERIOD" INTEGER,  
  "REF_AREA" TEXT=? AND  
  "INDICATOR" TEXT,  
  "ACTIVITY" TEXT,  
  "NUMBER_EMPL" TEXT,  
  "PRODUCT" TEXT,  
  "TURNOVER" TEXT,  
  "CLIENT_RESIDENCE" TEXT,  
  "UNIT_MEASURE" TEXT,  
  
  
3. These are the fields that will form the value (__group2__)

  "OBS_STATUS" TEXT,  
  "OBS_STATUS_1" TEXT,  
  "CONF_STATUS" TEXT,  
  "CONF_STATUS_1" TEXT,  
  "OBS_VALUE" REAL,  
  "UNIT_MULT" TEXT,  
  "DECIMALS" TEXT,  
  
  
4. I don't know what are these next fields
(maybe a real example will help, or asking the owner)
(for now I will add them as values)

  "DOMINANCE" TEXT,  
  "SHARE_SECOND" TEXT,  
  "COMMENT_OBS" TEXT  

## Structure:
1. Will be a table for keys:  
Flow1Keys(id primary key, unique(group1))

2. And a table for values:  
Flow1Values(timestamp, keyId, group2)
- timestamp is an INTEGER and there are some functions

## Operations:
1. Adding a row (group11, group21):
    - try to insert a key if do not exists (group11)
    - get the keyId for group11
    - add group21 to values if the latest timestamp for keyId has different values (group21 != group2(last timestamp))

2. Get the last value for a key
3. Get a value for a key before a timestamp (snapshot)
4. Get history for a key

## Some questions:
- What means a transaction in this context?
- Performance?
- Last values for a key and all others to be in different tables?

In [5]:
import sqlite3
import time


In [2]:
DB_FILE = 'solution1.db'
# Connect to a new database
with sqlite3.connect(DB_FILE) as conn:
    # Open the SQL script file
    with open('solution1.sql', 'r') as f:
        # Read the entire file contents
        sql_script = f.read()
        # Execute the script on the database connection
        conn.executescript(sql_script)


In [3]:
# now print the DB structure
# Connect to the database
with sqlite3.connect(DB_FILE) as conn:
    # Create a cursor object
    cursor = conn.cursor()
    # Query the database for a list of tables
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table' ORDER BY name;")
    tables = cursor.fetchall()
    # Iterate over each table and print its structure
    for table_name in tables:
        # Print the table name
        print(f'Table: {table_name[0]}')
        # Query the database for the columns in the table
        cursor.execute(f"PRAGMA table_info({table_name[0]})")
        columns = cursor.fetchall()
        # Print the columns and their properties
        for col in columns:
            print(f'  {col[1]} ({col[2]}) {"PRIMARY KEY" if col[5] == 1 else ""} {"NOT NULL" if col[3] == 1 else ""}')
        # Print a blank line to separate tables
        print('')

Table: flow1keys
  id (INTEGER) PRIMARY KEY 
  FREQ (TEXT(1))  
  TIME_PERIOD (INTEGER)  
  REF_AREA (TEXT(2))  
  INDICATOR (TEXT(4))  
  ACTIVITY (TEXT)  
  NUMBER_EMPL (TEXT)  
  PRODUCT (TEXT(2))  
  TURNOVER (TEXT(2))  
  CLIENT_RESIDENCE (TEXT(2))  
  UNIT_MEASURE (TEXT(3))  

Table: flow1values
  timestamp (INTEGER)  
  keyID (INTEGER)  
  OBS_STATUS (TEXT(1))  
  OBS_STATUS_1 (TEXT(1))  
  CONF_STATUS (TEXT(1))  
  CONF_STATUS_1 (TEXT(1))  
  OBS_VALUE (REAL)  
  UNIT_MULT (TEXT(1))  
  DECIMALS (TEXT(1))  
  DOMINANCE (TEXT)  
  SHARE_SECOND (TEXT)  
  COMMENT_OBS (TEXT)  



In [6]:
# 1. Operations: Adding a row
# function for insert, transactioned
def process_set(conn, data_set):
    # Create cursor
    c = conn.cursor()
    # Start a transaction
    c.execute('BEGIN TRANSACTION')
    for freq, time_period, ref_area, indicator, activity, number_empl, product, turnover, client_residence, unit_measure, unit_mult, decimals, \
        obs_status, obs_status_1, conf_status, conf_status_1, obs_value, dominance, share_second, comment_obs in data_set:
        # Convert (col1, ... colN) to keyID
        c.execute('SELECT id FROM flow1keys WHERE freq=? AND time_period=? AND ref_area=? AND indicator=? AND activity=? AND number_empl=? AND product=? AND turnover=? AND client_residence=? AND unit_measure=?', \
                  (freq, time_period, ref_area, indicator, activity, number_empl, product, turnover, client_residence, unit_measure))
        result = c.fetchone()
        if result:
            keyID = result[0]
        else:
            # If (col1, ... colN) doesn't exist in flow1keys, insert a new row and get the keyID
            c.execute('INSERT INTO flow1keys (freq, time_period, ref_area, indicator, activity, number_empl, product, turnover, client_residence, unit_measure) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?)', \
                      (freq, time_period, ref_area, indicator, activity, number_empl, product, turnover, client_residence, unit_measure))
            keyID = c.lastrowid

        # Check if the latest entry for the (keyID, value...) pair has a different value
        c.execute('SELECT obs_status, obs_status_1, conf_status, conf_status_1, obs_value, unit_mult, decimals, dominance, share_second, comment_obs FROM flow1values WHERE keyID=? ORDER BY timestamp DESC LIMIT 1', (keyID,))
        result = c.fetchone()
        if not result or \
            (result[0] != obs_status or result[1] != obs_status_1 or result[2] != conf_status or result[3] != conf_status_1 or result[4] != obs_value or result[5] != unit_mult):
            # Insert a new row in flow1values
            timestamp = int(time.time())
            c.execute('INSERT INTO flow1values (timestamp, keyID, obs_status, obs_status_1, conf_status, conf_status_1, obs_value, unit_mult, decimals, dominance, share_second, comment_obs) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)', \
                       (timestamp, keyID, obs_status, obs_status_1, conf_status, conf_status_1, obs_value, unit_mult, decimals, dominance, share_second, comment_obs))
    # Commit the transaction
    conn.commit()

# and a little example
dataset = (
    ('A', 2021, 'LT', 'ENT', 'B071', '_T', '_Z', '_Z', '_Z', 'BGN', '3', '2', 'P', 'L', 'M', 'T', 465784.79, '', '', ''),
    ('A', 2021, 'ME', 'EMPL', 'N', 'E20T49', '_Z', '_Z', '_Z', 'HUF', '3', '5', 'U', 'D', 'C', 'O', 866343.37, '', '', ''),
    ('A', 2020, 'HU', 'EMPL', 'J639', 'E2T9', '_Z', '_Z', '_Z', 'TRY', '0', '5', 'L', 'L', 'O', 'M', 973961.01, '', '', '')
)

# Connect to the database
with sqlite3.connect(DB_FILE) as conn:
    process_set(conn, dataset)


In [ ]:
# 1. Operations: Adding a row (but with pandas dataframe as datasource)
# function for insert, transactioned
import pandas as pd

def process_set(conn, df):
    c = conn.cursor()
    c.execute('BEGIN TRANSACTION')
    for _, row in df.iterrows():
        freq, time_period, ref_area, indicator, activity, number_empl, product, turnover, client_residence, unit_measure, unit_mult, decimals, obs_status, obs_status_1, conf_status, conf_status_1, obs_value, dominance, share_second, comment_obs = row.values
        c.execute('SELECT id FROM flow1keys WHERE freq=? AND time_period=? AND ref_area=? AND indicator=? AND activity=? AND number_empl=? AND product=? AND turnover=? AND client_residence=? AND unit_measure=?', \
                  (freq, time_period, ref_area, indicator, activity, number_empl, product, turnover, client_residence, unit_measure))
        result = c.fetchone()
        if result:
            keyID = result[0]
        else:
            c.execute('INSERT INTO flow1keys (freq, time_period, ref_area, indicator, activity, number_empl, product, turnover, client_residence, unit_measure) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?)', \
                      (freq, time_period, ref_area, indicator, activity, number_empl, product, turnover, client_residence, unit_measure))
            keyID = c.lastrowid

        c.execute('SELECT obs_status, obs_status_1, conf_status, conf_status_1, obs_value, unit_mult, decimals, dominance, share_second, comment_obs FROM flow1values WHERE keyID=? ORDER BY timestamp DESC LIMIT 1', (keyID,))
        result = c.fetchone()
        if not result or \
            (result[0] != obs_status or result[1] != obs_status_1 or result[2] != conf_status or result[3] != conf_status_1 or result[4] != obs_value or result[5] != unit_mult):
            timestamp = int(time.time())
            c.execute('INSERT INTO flow1values (timestamp, keyID, obs_status, obs_status_1, conf_status, conf_status_1, obs_value, unit_mult, decimals, dominance, share_second, comment_obs) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)', \
                       (timestamp, keyID, obs_status, obs_status_1, conf_status, conf_status_1, obs_value, unit_mult, decimals, dominance, share_second, comment_obs))
    conn.commit()

    # a small example with different data 
    dataset = pd.DataFrame((
        ('A', 2021, 'LT', 'ENT', 'B071', '_T', '_Z', '_Z', '_Z', 'BGN', '3', '2', 'P', 'L', 'M', 'T', 465784.79, '', '', ''),
    ('A', 2021, 'ME', 'EMPL', 'N', 'E20T49', '_Z', '_Z', '_Z', 'HUF', '3', '5', 'U', 'D', 'C', 'O', 866343.37, '', '', ''),
    ('A', 2020, 'HU', 'EMPL', 'J639', 'E2T9', '_Z', '_Z', '_Z', 'TRY', '0', '5', 'L', 'L', 'O', 'M', 973961.01, '', '', '')
)

In [11]:
# a little visualization of data in DB
with sqlite3.connect(DB_FILE) as conn:
    # Create a cursor object to execute SQL commands
    cur =conn.cursor()
    # Execute a SELECT statement to fetch all rows from the table
    cur.execute("SELECT * FROM flow1keys")
    # Fetch all rows and print them
    rows = cur.fetchall()
    print('flow1keys')
    if len(rows): 
        for row in rows:
            print(row)
    else:
        print('nothing to print')
    cur.execute("SELECT * FROM flow1values")
    # Fetch all rows and print them
    rows = cur.fetchall()
    print('flow1values')
    if len(rows): 
        for row in rows:
            print(row)
    else:
        print('nothing to print')
    cur.close()

flow1keys
(1, 'A', 2021, 'LT', 'ENT', 'B071', '_T', '_Z', '_Z', '_Z', 'BGN')
(2, 'A', 2021, 'ME', 'EMPL', 'N', 'E20T49', '_Z', '_Z', '_Z', 'HUF')
(3, 'A', 2020, 'HU', 'EMPL', 'J639', 'E2T9', '_Z', '_Z', '_Z', 'TRY')
flow1values
(1676630202, 1, 'P', 'L', 'M', 'T', 465784.79, '3', '2', '', '', '')
(1676630202, 2, 'U', 'D', 'C', 'O', 866343.37, '3', '5', '', '', '')
(1676630202, 3, 'L', 'L', 'O', 'M', 973961.01, '0', '5', '', '', '')
(1676630271, 1, 'P', 'L', 'M', 'T', 465750.33, '3', '2', '', '', '')


In [8]:
# try to add first one again
dataset = (
    ('A', 2021, 'LT', 'ENT', 'B071', '_T', '_Z', '_Z', '_Z', 'BGN', '3', '2', 'P', 'L', 'M', 'T', 465784.79, '', '', ''),
    )

# Connect to the database
with sqlite3.connect(DB_FILE) as conn:
    process_set(conn, dataset)


In [10]:
# now with a different obs_value
dataset = (
    ('A', 2021, 'LT', 'ENT', 'B071', '_T', '_Z', '_Z', '_Z', 'BGN', '3', '2', 'P', 'L', 'M', 'T', 465750.33, '', '', ''),
    )

# Connect to the database
with sqlite3.connect(DB_FILE) as conn:
    process_set(conn, dataset)


In [12]:
# 2. Operations: Get the last value for a key
# function for active data
def process_get(conn, key):
    # Create cursor
    c = conn.cursor()
    # unpack key
    freq, time_period, ref_area, indicator, activity, number_empl, product, turnover, client_residence, unit_measure = key
    # execute select to retrieve the key
    c.execute('SELECT id FROM flow1keys WHERE freq=? AND time_period=? AND ref_area=? AND indicator=? AND activity=? AND number_empl=? AND product=? AND turnover=? AND client_residence=? AND unit_measure=?', \
              (freq, time_period, ref_area, indicator, activity, number_empl, product, turnover, client_residence, unit_measure))
    result = c.fetchone()
    if result:
        keyID = result[0]
    else:
        raise Exception('No key for these values')
    # Get latest entry for the (keyID, value...) pair has a different value
    c.execute('SELECT obs_status, obs_status_1, conf_status, conf_status_1, obs_value, unit_mult, decimals, dominance, share_second, comment_obs FROM flow1values WHERE keyID=? ORDER BY timestamp DESC LIMIT 1', (keyID,))
    return c.fetchone()




In [14]:
# now some tests: positive
key = ('A', 2021, 'LT', 'ENT', 'B071', '_T', '_Z', '_Z', '_Z', 'BGN')
# Connect to the database
with sqlite3.connect(DB_FILE) as conn:
    print(process_get(conn, key))


('P', 'L', 'M', 'T', 465750.33, '3', '2', '', '', '')


In [15]:
# now some tests: negative - non existing key
key = ('A', 2021, 'RO', 'ENT', 'B071', '_T', '_Z', '_Z', '_Z', 'BGN')
# Connect to the database
with sqlite3.connect(DB_FILE) as conn:
    print(process_get(conn, key))


Exception: No key for these values

In [20]:
# 3. Operations: Get a value for a key before a timestamp (snapshot)
# function for snapshot data
# TODO: get and getsnapshot can be conbined by just adding an optional parameter of type date named snapshot
def process_getsnapshot(conn, key, date):
    # Create cursor
    c = conn.cursor()
    # unpack key
    freq, time_period, ref_area, indicator, activity, number_empl, product, turnover, client_residence, unit_measure = key
    # execute select to retrieve the key
    c.execute('SELECT id FROM flow1keys WHERE freq=? AND time_period=? AND ref_area=? AND indicator=? AND activity=? AND number_empl=? AND product=? AND turnover=? AND client_residence=? AND unit_measure=?', \
              (freq, time_period, ref_area, indicator, activity, number_empl, product, turnover, client_residence, unit_measure))
    result = c.fetchone()
    if result:
        keyID = result[0]
    else:
        raise Exception('No key for these values')
    # Get entry before (or equal) date for the (keyID, value...) pair has a different value
    c.execute('SELECT obs_status, obs_status_1, conf_status, conf_status_1, obs_value, unit_mult, decimals, dominance, share_second, comment_obs FROM flow1values WHERE keyID=? AND timestamp <=? ORDER BY timestamp DESC LIMIT 1', (keyID, date.timestamp()))
    return c.fetchone()


In [22]:
# positive test: the two records for the first key
from datetime import datetime

print('timestamp', current_time.timestamp())
key = ('A', 2021, 'LT', 'ENT', 'B071', '_T', '_Z', '_Z', '_Z', 'BGN')
# Connect to the database
with sqlite3.connect(DB_FILE) as conn:
    # equivalent with get: always when you ask with now() will be the last record
    date1 = datetime.now()
    print(process_getsnapshot(conn, key, date1))
    # tried to do a backward conversion from a value I know
    snaptime = 1676630270
    date2 = datetime.fromtimestamp(snaptime)
    print(process_getsnapshot(conn, key, date2))

timestamp 1676632644.542244
1676633081.445342
('P', 'L', 'M', 'T', 465750.33, '3', '2', '', '', '')
1676630270.0
('P', 'L', 'M', 'T', 465784.79, '3', '2', '', '', '')


In [23]:
# negative test: try to get a record before the first one
from datetime import datetime

key = ('A', 2021, 'LT', 'ENT', 'B071', '_T', '_Z', '_Z', '_Z', 'BGN')
# Connect to the database
with sqlite3.connect(DB_FILE) as conn:
    snaptime = 1676630200
    date3 = datetime.fromtimestamp(snaptime)
    print(process_getsnapshot(conn, key, date3))

1676630200.0
None


In [26]:
# 4. Operations: Get history for a key
# function for history data
# TODO: it can be combined with a start date (like in snapshot) and with another parameter limit to specify how many records
def process_gethistory(conn, key):
    # Create cursor
    c = conn.cursor()
    # unpack key
    freq, time_period, ref_area, indicator, activity, number_empl, product, turnover, client_residence, unit_measure = key
    # execute select to retrieve the key
    c.execute('SELECT id FROM flow1keys WHERE freq=? AND time_period=? AND ref_area=? AND indicator=? AND activity=? AND number_empl=? AND product=? AND turnover=? AND client_residence=? AND unit_measure=?', \
              (freq, time_period, ref_area, indicator, activity, number_empl, product, turnover, client_residence, unit_measure))
    result = c.fetchone()
    if result:
        keyID = result[0]
    else:
        raise Exception('No key for these values')
    # Get entry before (or equal) date for the (keyID, value...) pair has a different value
    c.execute('SELECT obs_status, obs_status_1, conf_status, conf_status_1, obs_value, unit_mult, decimals, dominance, share_second, comment_obs FROM flow1values WHERE keyID=? ORDER BY timestamp DESC', (keyID,))
    return c.fetchall()


In [27]:
# just a positive test for now
key = ('A', 2021, 'LT', 'ENT', 'B071', '_T', '_Z', '_Z', '_Z', 'BGN')
# Connect to the database
with sqlite3.connect(DB_FILE) as conn:
    print(process_gethistory(conn, key))

[('P', 'L', 'M', 'T', 465750.33, '3', '2', '', '', ''), ('P', 'L', 'M', 'T', 465784.79, '3', '2', '', '', '')]
